In [1]:
#======================================================================================================
# by kobongsoo, updata: 2022-04-13
#
# sentence-bert NLI 훈련 및 평가 예시
# => 기존 (distil)bert 모델을 가지고, NLI로 훈련 및 평가 후, S-BERT로 만드는 예시임.
#
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄일수 있음
#=> reduce_out_dimension = True 로 하면, 출력 임베딩 dimension이 줄어들게 설정가능함
#
# => sentence-transformers 패키지를 이용하여 구현 함.(*pip install -U sentence-transformers 설치 필요)
#
# 도큐먼트 : https://www.sbert.net/index.html
# 소스참고 : https://github.com/BM-K/KoSentenceBERT-ETRI

# pip install -U sentence-transformers
#======================================================================================================
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()
seed_everything(111)

logfilepath:bwdataset_2022-04-13.log
logfilepath:qnadataset_2022-04-13.log
logfilepath:../../log/s-bert_2022-04-13.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
import os

# s-bert로 만들 원본 bert 경로
model_name = "../../model/distilbert/distilbert-0331-TS-nli-0.1-10"

# 원본 bert를 sentencebert로 만든후 만들어진 s-bert 저장 경로
#smodel_path = 'output/training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
smodel_path = "../../model/sbert/sbert-distilbert-0331-TS-nli-0.1-10-nli-04-13"

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = "../../model/sbert/output"
os.makedirs(output_path, exist_ok=True)

train_file = '../../korpora/kornli/snli_1.0_train.ko.tsv'
eval_file = '../../korpora/korsts/tune_dev.tsv'
test_file = '../../korpora/korsts/tune_test.tsv'

train_batch_size = 32
num_epochs = 2

#============================================================================
# *출력 dimension을 줄일 경우에는 True로 하고, out_dimension에 줄일 값을 설정함
reduce_out_dimension = True  # True이면 dimension을 줄임=>Dense 모델 추가됨
out_dimension = 128
#============================================================================

# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_name, max_seq_length=128)

# word embedding_model 출력 
print(word_embedding_model)

Some weights of the model checkpoint at ../../model/distilbert/distilbert-0331-TS-nli-0.1-10 were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 


In [3]:
# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용
# pooling model 출력 
print(pooling_model)
print(pooling_model.get_sentence_embedding_dimension())

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
768


In [4]:
# 3. dense 모델 추가(옵션)
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄임.
#=> https://www.sbert.net/docs/training/overview.html?highlight=dense 참조
if reduce_out_dimension:
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), # 입력 dimension은 앞에 pooling모델 embedding dimension으로 지정
                               out_features=out_dimension,  # 출력 dimension
                               activation_function=nn.Tanh())  # activation function은 Tahn으로 정의

In [5]:
# SBERT 모델 생성
if reduce_out_dimension:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
else:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)


In [6]:
# 훈련 데이터 불러오기
# => [sentence1, sentence2], labels 식으로 만듬
logger.info("Read AllNLI train dataset")

label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_samples = []

with open(train_file, "rt", encoding="utf-8") as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, label = line.split('\t')
        label = label2int[label.strip()]
        train_samples.append(InputExample(texts=[s1, s2], label=label))

2022-04-13 13:33:15,869 - s-bert - INFO - Read AllNLI train dataset


In [7]:
# 데이터 셋, 데이터 로더, 손실함수 정의
train_dataset = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.SoftmaxLoss(model=model, 
                                sentence_embedding_dimension=model.get_sentence_embedding_dimension(), 
                                num_labels=len(label2int))

In [8]:
#Read STSbenchmark dataset and use it as development set
# 평가데이터 불러오기
#korsts 파일로 두 문장간 유사도를 수치로(5.0이 만점=매우 유사) 측정함.
logger.info("Read STSbenchmark dev dataset")
dev_samples = []

with open(eval_file, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        dev_samples.append(InputExample(texts= [s1,s2], label=score))

# 2개의 bert 모델에서 구한 2개의 embedding 값들의 cosine 유사도를 구해서, 이를 실제 score와 비교해서 유사도 측정함
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, 
                                                                 batch_size=train_batch_size, 
                                                                 name='sts-dev')

2022-04-13 13:33:18,463 - s-bert - INFO - Read STSbenchmark dev dataset


In [9]:

warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=smodel_path
          )


2022-04-13 13:33:18,503 - s-bert - INFO - Warmup-steps: 3439


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17193 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17193 [00:00<?, ?it/s]

In [10]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
##############################################################################
import time

test_samples = []
with open(test_file, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts=[s1,s2], label=score))

logger.info("\n")
logger.info("======================TEST===================")
logger.info("\n\n")
logger.info(f"model save path > {smodel_path}")
start = time.time()
model = SentenceTransformer(smodel_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test', show_progress_bar=True)
test_evaluator(model, output_path=output_path)
logger.info(f"처리시간 > {time.time() - start:.4f}")

2022-04-13 14:28:33,224 - s-bert - INFO - 

2022-04-13 14:28:33,225 - s-bert - INFO - ======================TEST===================
2022-04-13 14:28:33,225 - s-bert - INFO - 


2022-04-13 14:28:33,226 - s-bert - INFO - model save path > ../../model/sbert/sbert-distilbert-0331-TS-nli-0.1-10-nli-04-13


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

2022-04-13 14:28:36,026 - s-bert - INFO - 처리시간 > 2.7993
